In [1]:
import numpy as np
import pandas as pd
from scipy.fft import fft, ifft
from scipy.interpolate import interp1d
from scipy.stats import entropy, iqr, kurtosis, mode, skew
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import (Activation, BatchNormalization, Bidirectional,
                                     Conv1D, Conv2D, Dense, Dropout,
                                     Flatten, GlobalAveragePooling1D, LSTM,
                                     MaxPool1D, MaxPooling2D, Reshape,
                                     TimeDistributed)
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import os
import pickle
from pandas import read_csv, unique
from scipy.signal import argrelextrema, butter, find_peaks, lfilter, lfilter_zi
from sklearn2pmml import sklearn2pmml
from sklearn2pmml.pipeline import PMMLPipeline


In [2]:
X_respeck = pickle.load(open("X_respeck", "rb"))
y_respeck = pickle.load(open("y_respeck", "rb"))

In [3]:
X_respeck[0].shape

(50, 10)

In [4]:
X_train, X_temp, y_train, y_temp = train_test_split(X_respeck, y_respeck, test_size=0.2, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [5]:
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

enc = enc.fit(np.array(y_train))
y_train = enc.transform(y_train)
y_val = enc.transform(y_val)
y_test = enc.transform(y_test)



C:\Users\danin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [6]:
# model = Sequential()
# model.add(LSTM(32, return_sequences=True, input_shape=X_train[0].shape, activation='relu'))
# model.add(LSTM(32,return_sequences=True, activation='relu'))
# model.add(Conv1D(filters=64,kernel_size=2, activation='relu', strides=2))
# model.add(MaxPool1D(pool_size=4, padding='same'))
# model.add(Conv1D(filters=192, kernel_size=2, activation='relu', strides=1))
# model.add(GlobalAveragePooling1D())
# model.add(BatchNormalization(epsilon=1e-06))
# model.add(Dense(12))
# model.add(Activation('softmax'))

# print(model.summary())



# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
# history = model.fit(
#     X_train, y_train,
#     epochs=10,
#     batch_size=32,
#     validation_split=0.1,
#     shuffle=False
# )

In [8]:
# print(X_val[1].reshape(500).shape)
# predictions = model.predict(X_val)
# print(predictions[0])
# predictions = np.argmax(predictions, axis=1)
# # for i, p in enumerate(predictions):
# #     if p == 10 or p == 11:
# #         p = sit_stand_tree.predict(X_val[i].reshape(500).reshape(1,-1))
# y_val_pred = np.argmax(y_val, axis=1)
# cm = confusion_matrix(y_val_pred, predictions)
# cm_disp = ConfusionMatrixDisplay(confusion_matrix= cm)
# cm_disp.plot()
# plt.show()

In [9]:
# Define the Random Forest model
# rf_model = RandomForestClassifier(random_state=42)

# rf_model_pipeline = PMMLPipeline([
# 	("classifier", RandomForestClassifier(random_state=42))
# ])

# X_train_rf = X_train.reshape(X_train.shape[0], -1)
# # Train the model
# rf_model_pipeline.fit(X_train_rf, y_train)

In [10]:

# Define the Random Forest model
rf_model = RandomForestClassifier(random_state=42)

rf_model_pipeline = rf_model
# rf_model_pipeline = PMMLPipeline([
# 	("classifier", RandomForestClassifier(random_state=42))
# ])

X_train_rf = X_train.reshape(X_train.shape[0], -1)
# Train the model
rf_model_pipeline.fit(X_train_rf, y_train)



# create individual models
cnn_lstm = Sequential()
cnn_lstm.add(LSTM(32, return_sequences=True, input_shape=X_train[0].shape, activation='relu'))
cnn_lstm.add(LSTM(32,return_sequences=True, activation='relu'))
cnn_lstm.add(Conv1D(filters=64,kernel_size=2, activation='relu', strides=2))
cnn_lstm.add(MaxPool1D(pool_size=4, padding='same'))
cnn_lstm.add(Conv1D(filters=192, kernel_size=2, activation='relu', strides=1))
cnn_lstm.add(GlobalAveragePooling1D())
cnn_lstm.add(BatchNormalization(epsilon=1e-06))
cnn_lstm.add(Dense(12))
cnn_lstm.add(Activation('softmax'))
cnn_lstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
cnn_lstm.fit(X_train, y_train, epochs=20, batch_size=32, verbose=1)


x_val_rf = X_val.reshape(X_val.shape[0], -1)

KeyboardInterrupt: 

In [ ]:

# Evaluate the model
print("Evaluate the model:")
# rf_results = rf_model.evaluate(x=x_val_rf, y=y_val)
cnn_lstm_results = cnn_lstm.evaluate(x=X_val, y=y_val)

# show confusion matrix
y_pred_cnn_lstm = cnn_lstm.predict(X_val)
y_pred_rf = rf_model_pipeline.predict(x_val_rf)
y_pred_ensemble = np.mean(np.array([y_pred_cnn_lstm, y_pred_rf]), axis=0)
y_pred_ensemble = np.argmax(y_pred_ensemble, axis=1)
y_pred_cnn_lstm = np.argmax(y_pred_cnn_lstm, axis=1)
y_pred_rf = np.argmax(y_pred_rf, axis=1)

cm = confusion_matrix(np.argmax(y_val, axis =1), y_pred_ensemble)
cm_disp = ConfusionMatrixDisplay(confusion_matrix= cm)
cm_disp.plot()
plt.show()

In [ ]:
def accuracy(y_true, y_pred): 
    correct_predictions = np.sum(y_true == y_pred)
    total_predictions = len(y_true)
    accuracy = correct_predictions / total_predictions
    print(f"Accuracy: {accuracy}")
    return accuracy
accuracy(y_pred_ensemble, np.argmax(y_val, axis =1))

In [ ]:
# # Define the task configuration for binary classification

# # Create the RandomForestModel
# forest = tfdf.keras.RandomForestModel( random_seed=42)
# # Train the model
# history = forest.fit(
#     np.array([x.reshape(x.shape[0], -1) for x in X_train])
# , np.argmax(y_train, axis= 1),
#     epochs=1,
#     batch_size=32,
#     validation_split=0.1,
# )

# # Evaluate the model
# evaluation = forest.evaluate(X_val, y_val)
# print(evaluation)

In [ ]:
predictions = rf_model_pipeline.predict(np.array(X_val).reshape(np.array(X_val).shape[0], -1))
y_val_pred = np.argmax(y_val, axis=1)
predictions = np.argmax(predictions, axis=1)
cm = confusion_matrix(y_val_pred, predictions)
cm_disp = ConfusionMatrixDisplay(confusion_matrix= cm)
cm_disp.plot()
plt.show()

In [ ]:
def get_select_actions(actions, X, y):
    filtered_y = []
    filtered_X = []
    for j, elm in enumerate(y):
        for i in actions:
            if elm[i] == 1:
                onehot = np.zeros(len(actions))
                onehot[actions.index(i)] = 1
                filtered_y.append(onehot)
                filtered_X.append(pd.DataFrame(X[j]))
    return filtered_X, np.array(filtered_y)


In [ ]:
stand_sit_X, stand_sit_y = get_select_actions([10,11], X_train, y_train)
stand_sit_X_tree = np.array(stand_sit_X).reshape(np.array(stand_sit_X).shape[0], -1)
stand_sit_y = np.argmax(stand_sit_y, axis=1)

# build random decision tree model
sit_stand_Pipeline = PMMLPipeline([
	("classifier", RandomForestClassifier(random_state=42))
])

# Train the model
# rf_model_pipeline.fit(X_train_rf, y_train)
sit_stand_Pipeline.fit(stand_sit_X_tree, stand_sit_y)

In [ ]:
stand_sit_X_val, stand_sit_y_val = get_select_actions([10,11], X_val, y_val)

predictions = sit_stand_Pipeline.predict(np.array(stand_sit_X_val).reshape(np.array(stand_sit_X_val).shape[0], -1))
stand_sit_y_val = np.argmax(stand_sit_y_val, axis=1)

cm = confusion_matrix(stand_sit_y_val, predictions)
cm_disp = ConfusionMatrixDisplay(confusion_matrix= cm)
cm_disp.plot()
plt.show()

In [ ]:
# rf_stand_sit_model = tfdf.keras.RandomForestModel(num_trees=1000)
# stand_sit_X, stand_sit_y = get_select_actions([10,11], X_train, y_train)
# X_stand_sit_train_rf = np.array(stand_sit_X).reshape(np.array(stand_sit_X).shape[0], -1)
# # Train the model
# rf_stand_sit_model.compile(metrics=["precision"])
# rf_stand_sit_model.fit(X_stand_sit_train_rf, np.argmax(stand_sit_y, axis = 1), verbose=1)

In [ ]:
# predictions = rf_stand_sit_model.predict(np.array(stand_sit_X_val).reshape(np.array(stand_sit_X_val).shape[0], -1))
# predictions = np.argmax(predictions, axis=1)
# cm = confusion_matrix(stand_sit_y_val, predictions)
# cm_disp = ConfusionMatrixDisplay(confusion_matrix= cm)
# cm_disp.plot()
# plt.show()

In [ ]:
np.array(stand_sit_X_val).shape

In [ ]:
# build model to test if the user is sitting or standing - actually bad at sit stand detection
# model_standing = Sequential()
# model_standing.add(LSTM(32, return_sequences=True, input_shape=stand_sit_X[0].shape, activation='relu'))
# model_standing.add(LSTM(32,return_sequences=True, activation='relu'))
# model_standing.add(Conv1D(filters=64,kernel_size=2, activation='relu', strides=2))
# model_standing.add(MaxPool1D(pool_size=4, padding='same'))
# model_standing.add(Conv1D(filters=192, kernel_size=2, activation='relu', strides=1))
# model_standing.add(GlobalAveragePooling1D())
# model_standing.add(BatchNormalization(epsilon=1e-06))
# model_standing.add(Dense(2))
# model_standing.add(Activation('softmax'))

# print(stand_sit_X[0].shape)
# model_standing.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# history = model_standing.fit(
#     stand_sit_X, stand_sit_y,
#     epochs=10,
#     batch_size=32,
#     validation_split=0.1,
#     shuffle=False
# )

In [ ]:
# X_test_tree = X_test.reshape(X_val.shape[0], -1)
# # evaluate tree on test set
# tree_test_acc = tree.score(X_test_tree, y_val)
# print(tree_test_acc)
# predictions = tree.predict(X_val.reshape(X_val.shape[0], -1))
# predictions = np.argmax(predictions, axis=1)
# y_val_pred = np.argmax(y_val, axis=1)
# cm = confusion_matrix(y_val_pred, predictions)
# cm_disp = ConfusionMatrixDisplay(confusion_matrix= cm)
# cm_disp.plot()
# plt.show()

In [ ]:
predictions = cnn_lstm.predict(X_val)
predictions = np.argmax(predictions, axis=1)
for i, p in enumerate(predictions):
    if p == 10 or p == 11:
        predictions[i] = 10 + sit_stand_Pipeline.predict(X_val[i].reshape(X_val.shape[1] * X_val.shape[2]).reshape(1,-1))
y_val_pred = np.argmax(y_val, axis=1)
cm = confusion_matrix(y_val_pred, predictions)
cm_disp = ConfusionMatrixDisplay(confusion_matrix= cm)
cm_disp.plot()
plt.show()

In [ ]:
print(classification_report(y_val_pred, predictions))
accuracy(predictions, np.argmax(y_val, axis =1))

In [ ]:
cnn_lstm.evaluate(X_val, y_val)

In [ ]:
# Evaluate the model
print("Evaluate the model:")
# rf_results = rf_model_pipeline.evaluate(x=x_val_rf, y=y_val)
# cnn_lstm_results = cnn_lstm.evaluate(x=X_val, y=y_val)
# print (rf_results)
def full_stack_model(X, alpha):
    # show confusion matrix
    X_rf = X.reshape(X.shape[0], -1)
    y_pred_cnn_lstm = alpha*cnn_lstm.predict(X)
    y_pred_rf = rf_model_pipeline.predict(X_rf)
    y_pred_ensemble = np.mean(np.array([y_pred_cnn_lstm, y_pred_rf]), axis=0)
    y_pred_ensemble = np.argmax(y_pred_ensemble, axis=1)
    y_pred_cnn_lstm = np.argmax(y_pred_cnn_lstm, axis=1)
    y_pred_rf = np.argmax(y_pred_rf, axis=1)

    for i, p in enumerate(y_pred_ensemble):
        if p == 10 or p == 11:
            y_pred_ensemble[i] = y_pred_rf[i] # = 10 + sit_stand_Pipeline.predict(X[i].reshape(X.shape[1] * X.shape[2]).reshape(1,-1))
        if y_pred_ensemble[i] == 0:
            y_pred_ensemble[i] = y_pred_cnn_lstm[i]
    return y_pred_ensemble 
        
y_pred_ensemble = full_stack_model(X_val,1)
cm = confusion_matrix(np.argmax(y_val, axis =1), y_pred_ensemble)
cm_disp = ConfusionMatrixDisplay(confusion_matrix= cm)
cm_disp.plot()
plt.show()
accuracy(y_pred_ensemble, np.argmax(y_val, axis =1))

Evaluate the model:


NameError: name 'X_val' is not defined

In [ ]:
max_ = 0
max_alpha = 0
for alpha in np.arange(0, 2, 0.1):
    y_pred_ensemble = full_stack_model(X_test, alpha)
    print(alpha)
    acc = (accuracy(y_pred_ensemble, np.argmax(y_test, axis =1)))
    if acc > max_:
        max_ = acc
        max_alpha = alpha
print(max_alpha)

In [ ]:
def full_stack_model(X):
    # show confusion matrix
    X_rf = X.reshape(X.shape[0], -1)
    y_pred_cnn_lstm = 1.3*cnn_lstm.predict(X)
    y_pred_rf = rf_model_pipeline.predict(X_rf)
    y_pred_ensemble = np.mean(np.array([y_pred_cnn_lstm, y_pred_rf]), axis=0)
    y_pred_ensemble = np.argmax(y_pred_ensemble, axis=1)
    # correct for known issues with sit stand detection
    for i, p in enumerate(y_pred_ensemble):
        if p == 10 or p == 11:
            y_pred_ensemble[i] = np.argmax(y_pred_rf[i], axis =1)
        if y_pred_ensemble[i] == 0:
            y_pred_ensemble[i] = np.argmax(y_pred_cnn_lstm[i], axis =1)
    return y_pred_ensemble 
y_pred_ensemble = full_stack_model(X_test)

In [ ]:
# accuracy(full_stack_model(X_test), np.argmax(y_test, axis =1))

cm = confusion_matrix(np.argmax(y_test, axis=1), y_pred_ensemble)
cm_disp = ConfusionMatrixDisplay(confusion_matrix= cm)
cm_disp.plot()
plt.show()
accuracy(y_pred_ensemble, np.argmax(y_test, axis =1))

In [ ]:
# for w in [25,50,75,100,125,150]:    
#     X_respeck = pickle.load(open("X_respeck_" + str(w), "rb"))
#     y_respeck = pickle.load(open("y_respeck_" + str(w), "rb"))

#     X_train, X_temp, y_train, y_temp = train_test_split(X_respeck, y_respeck, test_size=0.2, random_state=42)
#     X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

#     enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

#     enc = enc.fit(np.array(y_train))
#     y_train = enc.transform(y_train)
#     y_val = enc.transform(y_val)
#     y_test = enc.transform(y_test)


#     # Define the Random Forest model
#     rf_model = RandomForestClassifier(random_state=42)
#     X_train_rf = X_train.reshape(X_train.shape[0], -1)
#     # Train the model
#     rf_model.fit(X_train_rf, y_train)

#     # create individual models
#     cnn_lstm = Sequential()
#     cnn_lstm.add(LSTM(32, return_sequences=True, input_shape=X_train[0].shape, activation='relu'))
#     cnn_lstm.add(LSTM(32,return_sequences=True, activation='relu'))
#     cnn_lstm.add(Conv1D(filters=64,kernel_size=2, activation='relu', strides=2))
#     cnn_lstm.add(MaxPool1D(pool_size=4, padding='same'))
#     cnn_lstm.add(Conv1D(filters=192, kernel_size=2, activation='relu', strides=1))
#     cnn_lstm.add(GlobalAveragePooling1D())
#     cnn_lstm.add(BatchNormalization(epsilon=1e-06))
#     cnn_lstm.add(Dense(12))
#     cnn_lstm.add(Activation('softmax'))
#     cnn_lstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#     cnn_lstm.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)


#     stand_sit_X, stand_sit_y = get_select_actions([10,11], X_train, y_train)
#     stand_sit_X_tree = np.array(stand_sit_X).reshape(np.array(stand_sit_X).shape[0], -1)
#     stand_sit_y = np.argmax(stand_sit_y, axis=1)

#     # build random decision tree model
#     sit_stand_tree = RandomForestClassifier(random_state=42)
#     sit_stand_tree.fit(stand_sit_X_tree, stand_sit_y)


#     def full_stack_model(X):
#         # show confusion matrix
#         X_rf = X.reshape(X.shape[0], -1)
#         y_pred_cnn_lstm = 0.7*cnn_lstm.predict(X)
#         y_pred_rf = rf_model.predict(X_rf)
#         y_pred_ensemble = np.mean(np.array([y_pred_cnn_lstm, y_pred_rf]), axis=0)
#         y_pred_ensemble = np.argmax(y_pred_ensemble, axis=1)
#         y_pred_cnn_lstm = np.argmax(y_pred_cnn_lstm, axis=1)
#         y_pred_rf = np.argmax(y_pred_rf, axis=1)

#         for i, p in enumerate(y_pred_ensemble):
#             if p == 10 or p == 11:
#                 y_pred_ensemble[i] = 10 + sit_stand_tree.predict(X[i].reshape(X.shape[1] * X.shape[2]).reshape(1,-1))
#         return y_pred_ensemble 
#     y_pred_ensemble = full_stack_model(X_test)

#     cm = confusion_matrix(np.argmax(y_test, axis=1), y_pred_ensemble)
#     cm_disp = ConfusionMatrixDisplay(confusion_matrix= cm)
#     cm_disp.plot()
#     plt.show()
#     accuracy(y_pred_ensemble, np.argmax(y_test, axis =1))